In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/place-review-dataset-niche-usa/niche_review_count.json
/kaggle/input/place-review-dataset-niche-usa/niche_entity_name.json
/kaggle/input/place-review-dataset-niche-usa/niche_part1.json
/kaggle/input/place-review-dataset-niche-usa/niche_entity_details.json
/kaggle/input/place-review-dataset-niche-usa/niche_part2.json


In [2]:
import pandas as pd
import json

In [3]:
code = """
def say_hello():
    print("Hello from the saved Python file!")

if __name__ == "__main__":
    say_hello()
"""

# Save code to a .py file
with open('/kaggle/working/my_script.py', 'w') as file:
    file.write(code)
!python /kaggle/working/my_script.py

Hello from the saved Python file!


In [4]:

# Load JSON data
with open('/kaggle/input/place-review-dataset-niche-usa/niche_entity_details.json', 'r') as f:
    data = json.load(f)

# Initialize a list to store each row as a dictionary
flattened_data = []

# Iterate through each entity in the JSON data correctly using `data.items()`
for guid, details in data.items():
    row = {
        'guid': guid,
        'badge_display': details.get('badge', {}).get('display'),
        'badge_ordinal': details.get('badge', {}).get('ordinal'),
        'badge_total': details.get('badge', {}).get('total'),
        'badge_vanityURL': details.get('badge', {}).get('vanityURL'),
        'badge_photoURL_desktop': details.get('badge', {}).get('photoURLs', {}).get('desktop'),
        'badge_photoURL_mobile': details.get('badge', {}).get('photoURLs', {}).get('mobile'),
        'location_lat': details.get('content', {}).get('centroid', {}).get('lat'),
        'location_lon': details.get('content', {}).get('centroid', {}).get('lon'),
        'entity_name': details.get('content', {}).get('entity', {}).get('name'),
        'entity_location': details.get('content', {}).get('entity', {}).get('location'),
        'entity_type': details.get('content', {}).get('entity', {}).get('type'),
        'entity_genus': details.get('content', {}).get('entity', {}).get('genus'),
        'entity_population': next((fact['value'] for fact in details.get('content', {}).get('facts', []) if fact.get('label') == "Population"), None)
    }
    
    # Add grades, if available
    grades = details.get('content', {}).get('grades', [])
    for grade in grades:
        row[f'grade_{grade["label"].replace(" ", "_")}'] = grade.get('value')
    
    # Add featured review information if available
    featured_review = details.get('content', {}).get('featuredReview', {})
    row['featured_review_author'] = featured_review.get('author')
    row['featured_review_body'] = featured_review.get('body')
    row['featured_review_rating'] = featured_review.get('rating')
    row['featured_review_created'] = featured_review.get('created')
    
    # Append the flattened row to the list
    flattened_data.append(row)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(flattened_data)

# Save the DataFrame to CSV
df.to_csv('niche_entity_details_flat.csv', index=False)

print("Data has been flattened and saved to 'niche_entity_details_flat.csv'")


Data has been flattened and saved to 'niche_entity_details_flat.csv'


In [5]:

print(df.info())  # Check the structure and types of columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50253 entries, 0 to 50252
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   guid                       50253 non-null  object 
 1   badge_display              11948 non-null  object 
 2   badge_ordinal              11948 non-null  float64
 3   badge_total                11948 non-null  float64
 4   badge_vanityURL            11948 non-null  object 
 5   badge_photoURL_desktop     11948 non-null  object 
 6   badge_photoURL_mobile      11948 non-null  object 
 7   location_lat               50183 non-null  float64
 8   location_lon               50183 non-null  float64
 9   entity_name                50253 non-null  object 
 10  entity_location            50253 non-null  object 
 11  entity_type                50253 non-null  object 
 12  entity_genus               50253 non-null  object 
 13  entity_population          50253 non-null  int

In [6]:
with open('/kaggle/input/place-review-dataset-niche-usa/niche_entity_name.json', 'r') as f:
    data = json.load(f)

# Convert JSON to a list of dictionaries with 'guid' and 'name' columns
flattened_data = [{'guid': guid, 'name': name} for guid, name in data.items()]

# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(flattened_data)

# Save DataFrame to CSV
df.to_csv('niche_entity_name_flat.csv', index=False)

print("Data has been saved to 'niche_entity_name_flat.csv'")


Data has been saved to 'niche_entity_name_flat.csv'


In [7]:
print(df.head())

                                   guid                      name
0  df1e4fd2-772c-4c38-94e5-4fdf7b969398              Chesterbrook
1  382e8519-b5c6-4658-bf30-5592d5aeb699                  Bluemont
2  eb3305be-9dac-4bc9-9c6f-1229a2019d31  Radnor/Fort Myer Heights
3  c65a22ea-39da-412e-9b06-f161a23b9da9               Holly Hills
4  2894f360-1545-48ef-a77d-eb3d777501cf                Penn Wynne


In [8]:
import pandas as pd
import json

# Load JSON data
with open('/kaggle/input/place-review-dataset-niche-usa/niche_part1.json', 'r') as f:
    data = json.load(f)

# Initialize a list to store each review as a row in a dictionary
flattened_data = []

# Iterate through each place's reviews in the JSON data
for place_guid, reviews in data.items():
    # Each place may have multiple reviews
    for review in reviews:
        # Create a dictionary to store the review information
        row = {
            'place_guid': place_guid,
            'review_guid': review.get('guid'),
            'review_body': review.get('body'),
            'rating': review.get('rating'),
            'author': review.get('author'),
            'created': review.get('created'),
            # Join categories list into a single string if multiple categories are present
            'categories': ", ".join(review.get('categories', []))
        }
        
        # Append the row to the list
        flattened_data.append(row)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(flattened_data)

# Save the DataFrame to CSV
df.to_csv('training_data.csv', index=False)

print("Data has been flattened and saved to 'training_data.csv'")


Data has been flattened and saved to 'training_data.csv'


In [9]:
print(df.head())

                             place_guid                           review_guid  \
0  76c89487-bffb-4e28-a402-6f82996e319d  e9262a21-caae-4ee7-b127-4d885ec89669   
1  76c89487-bffb-4e28-a402-6f82996e319d  71967f5b-6573-4398-8718-41ff10cb5a98   
2  76c89487-bffb-4e28-a402-6f82996e319d  5590a4a3-83e4-44a4-b85c-86b9d981395f   
3  76c89487-bffb-4e28-a402-6f82996e319d  0286dce9-6178-4c0f-8b44-0fc479795c1f   
4  76c89487-bffb-4e28-a402-6f82996e319d  fafea388-f41c-4327-b6d7-cf62afe17b08   

                                         review_body  rating  \
0  Roxbury an average place it is dangerous at  t...       3   
1  I like how everything is close by and nothing ...       5   
2  I have lived in the Roxbury area for more than...       4   
3  Depending on whether the part of Roxbury is mo...       2   
4  For what it's worth, I would say the only thin...       3   

             author                      created          categories  
0  Current Resident   2020-12-18T16:59:33.28511Z  Overall

In [10]:
import pandas as pd
import json

# Load JSON data
with open('/kaggle/input/place-review-dataset-niche-usa/niche_part2.json', 'r') as f:
    data = json.load(f)

# Initialize a list to store each review as a row in a dictionary
flattened_data = []

# Iterate through each place's reviews in the JSON data
for place_guid, reviews in data.items():
    # Each place may have multiple reviews
    for review in reviews:
        # Create a dictionary to store the review information
        row = {
            'place_guid': place_guid,
            'review_guid': review.get('guid'),
            'review_body': review.get('body'),
            'rating': review.get('rating'),
            'author': review.get('author'),
            'created': review.get('created'),
            # Join categories list into a single string if multiple categories are present
            'categories': ", ".join(review.get('categories', []))
        }
        
        # Append the row to the list
        flattened_data.append(row)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(flattened_data)

# Save the DataFrame to CSV
df.to_csv('training_data2.csv', index=False)

print("Data has been flattened and saved to 'training_data2.csv'")
print(df.head())

Data has been flattened and saved to 'training_data2.csv'
                             place_guid                           review_guid  \
0  0002941f-8dbf-4b2e-83aa-7b27ddab05ba  392c6f1d-cb59-4ca7-9e6b-69fdd6bc0f1a   
1  0002941f-8dbf-4b2e-83aa-7b27ddab05ba  758db7c6-adff-4b2e-96c1-96addf039b13   
2  0002941f-8dbf-4b2e-83aa-7b27ddab05ba  29d80034-73d7-44f1-b6c8-aff790ba3028   
3  0002941f-8dbf-4b2e-83aa-7b27ddab05ba  ba532f4f-7e73-42ec-ab72-beeff6926d8a   
4  0002941f-8dbf-4b2e-83aa-7b27ddab05ba  9cb0061e-14f9-4473-8f77-1f7271b29a6f   

                                         review_body  rating  \
0  Waxahachie is a very friendly and safe place t...       4   
1  Lived here my whole life. Town is very friendl...       5   
2  I enjoy Waxahachie. The one thing I would love...       4   
3  They have a wide variety of restaurants and st...       4   
4  If you only look at and notice the exteriors o...       1   

             author                      created          categories  

In [11]:
import pandas as pd
import json

# Load JSON data
with open('/kaggle/input/place-review-dataset-niche-usa/niche_review_count.json', 'r') as f:
    data = json.load(f)

# Convert JSON to a list of dictionaries with 'guid' and 'review_count' columns
flattened_data = [{'guid': guid, 'review_count': count} for guid, count in data.items()]

# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(flattened_data)

# Save DataFrame to CSV
df.to_csv('review_count.csv', index=False)

print("Data has been saved to 'review_count.csv'")
print(df.head())

Data has been saved to 'review_count.csv'
                                   guid  review_count
0  76c89487-bffb-4e28-a402-6f82996e319d           114
1  84fe50d6-43be-438a-98b1-db76a4320673             5
2  86282b3f-1e86-4a84-9dd1-b53de90b1678             6
3  5b6b16b4-05c4-4da2-b899-5a991d89ab5a            13
4  9d20dd4a-34b8-4b59-a2d9-9089c2b86f41            48
